In [2]:
import requests
import json
import os

# Remember stop isn't inclusive, so we need to add 1 to the stop value, for example getting Gen 1 would be start=1, stop=152
def fetch_pokemon_from_api(start, stop):

    # This is the function that is fetching data directly from the PokeAPI, please only call this function if the JSON file doesn't exist or is having problems.
    # Think of this function from here on out as a helper function to the main function that is essentially just grabbing the data.
    # In short we will be using fetch_ from here on out as the helper function and get_ as the main function for retrieving data.

    pokemon_list = []
    base_url = "https://pokeapi.co/api/v2/pokemon"
    
    
    for pokemon_id in range(start, stop):  # Grab Pokemon In Given Range
        try:
			# construct the url for call
            url = f"{base_url}/{pokemon_id}"
			# make the call, adding a timeout condition
            response = requests.get(url, timeout=15)
            
            # successfull call
            if response.status_code == 200:
                data = response.json()
                
                # Extract abilities
                abilities = []
                for ability in data.get('abilities', []):
                    abilities.append({
                        'name': ability['ability']['name'],
                        'is_hidden': ability['is_hidden'],
                        'slot': ability['slot']
                    })
                
                # Extract types
                types = [type_info['type']['name'] for type_info in data.get('types', [])]
                
                stats = []
                
                for stat in data.get('stats', []):
                    stats.append({
                        'name': stat['stat']['name'],
                        'base_stat': stat['base_stat'],
                        'effort': stat['effort']
                    })
                
                
                
                # Get image URLs
                sprites = data.get('sprites', {})
                images = {
                    'official_artwork': sprites.get('other', {}).get('official-artwork', {}).get('front_default'),
                    'front_default': sprites.get('front_default'),
                    'front_shiny': sprites.get('front_shiny')
                }
                
                # Create Pokemon data struct
                pokemon_data = {
                    'id': data['id'],
                    'name': data['name'].capitalize(), # capitalize the first letter
                    'abilities': abilities,  
                    'types': types,
                    'images': images,
                    'height': data['height'],
                    'weight': data['weight'],
                    'base_experience': data['base_experience'],
                    'stats': stats
                }
                
                # adding struct to the array of pokemon
                pokemon_list.append(pokemon_data)
				
				# print successful capture
                print(f"Fetched {pokemon_data['name']} (ID: {pokemon_id})")
                
            else:
				# debug failure
                print(f"Failed to fetch Pokemon ID {pokemon_id}")
                
        except requests.RequestException as e:
            print(f"Error fetching Pokemon ID {pokemon_id}: {e}")
            continue
    
    return pokemon_list

def write_pokemon_toJson(gen_number, pokemon_list):
    filename = f"gen{gen_number}output.json"
    with open(filename, 'w') as f:
        json.dump(pokemon_list, f, indent=4)


In [ ]:
import time

count = 1
while count <= 9:
    start = int(input("Enter the start number: "))
    stop = int(input("Enter the stop number: "))
    pokemon_list = fetch_pokemon_from_api(start, stop)
    time.sleep(5) # <-- Waiting 5 seconds to try and not get rate limited or anything, haven't fully read the docs yet, so not sure if this is needed or not.
    write_pokemon_toJson(count, pokemon_list)
    count += 1


